In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, classification_report
import time

In [3]:
# Define parameters
base_path = "D:\CNN Projects\Breast Cancer\BUSI"
img_height = 227
img_width = 227
num_classes = 3
batch_size = 32
epochs = 50

In [4]:
# Function to resize images
def resize_image(image_path, img_height, img_width):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
    return resized_image

# Function to get dataset and labels
def get_dataset(data_path, img_height, img_width):
    dataset = []
    labels = []

    class_map = {
        'normal': 0,
        'benign': 1,
        'malignant': 2,
    }

    for class_name in class_map.keys():
        class_dir = os.path.join(data_path, class_name)
        if not os.path.isdir(class_dir):
            continue

        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                label = class_map[class_name]
                labels.append(label)
                img = resize_image(os.path.join(class_dir, filename), img_height, img_width)
                dataset.append(img)

    # Convert to numpy arrays
    dataset = np.array(dataset)
    labels = np.array(labels)

    return dataset, labels

In [6]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Define function to split dataset
def split_dataset(dataset, labels, test_size=0.3, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# Load dataset and labels
dataset, labels = get_dataset(base_path, img_height, img_width)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = split_dataset(dataset, labels, test_size=0.3, random_state=42)

# Normalize pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


### Training without data augmentation

In [7]:
# Define the model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'))

# Convolutional layer 4
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'))

# Convolutional layer 5
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Flatten the output from convolutional layers
model.add(Flatten())

# Dense layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Dense layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()

# Train model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Calculate training time
training_time = time.time() - start_time
print("\nTraining time: {:.2f} seconds".format(training_time))



Epoch 1/50


18/18 [==============================] - 77s 4s/step - loss: 0.9821 - accuracy: 0.5385 - val_loss: 0.9603 - val_accuracy: 0.5489
Epoch 2/50
18/18 [==============================] - 72s 4s/step - loss: 0.9418 - accuracy: 0.5641 - val_loss: 0.9378 - val_accuracy: 0.5489
Epoch 3/50
18/18 [==============================] - 67s 4s/step - loss: 0.8982 - accuracy: 0.5678 - val_loss: 1.1475 - val_accuracy: 0.5489
Epoch 4/50
18/18 [==============================] - 83s 5s/step - loss: 0.9141 - accuracy: 0.6081 - val_loss: 0.9055 - val_accuracy: 0.6085
Epoch 5/50
18/18 [==============================] - 71s 4s/step - loss: 0.8337 - accuracy: 0.6392 - val_loss: 0.8546 - val_accuracy: 0.6553
Epoch 6/50
18/18 [==============================] - 66s 4s/step - loss: 0.7772 - accuracy: 0.6502 - val_loss: 0.8210 - val_accuracy: 0.6511
Epoch 7/50
18/18 [==============================] - 65s 4s/step - loss: 0.7147 - accuracy: 0.6630 - val_loss: 0.8042 - val_accuracy: 0.6638
Epoch 8/50
18/18

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create an instance of ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

### Training with data augmentation

In [14]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test)
)

Epoch 1/50
17/17 [==============================] - 70s 4s/step - loss: 0.6199 - accuracy: 0.7418 - val_loss: 0.6945 - val_accuracy: 0.8128
Epoch 2/50
17/17 [==============================] - 69s 4s/step - loss: 0.6259 - accuracy: 0.7363 - val_loss: 0.6671 - val_accuracy: 0.7702
Epoch 3/50
17/17 [==============================] - 74s 4s/step - loss: 0.5664 - accuracy: 0.7308 - val_loss: 0.7108 - val_accuracy: 0.8043
Epoch 4/50
17/17 [==============================] - 73s 4s/step - loss: 0.5874 - accuracy: 0.7198 - val_loss: 0.6684 - val_accuracy: 0.8085
Epoch 5/50
17/17 [==============================] - 69s 4s/step - loss: 0.5894 - accuracy: 0.7418 - val_loss: 0.6496 - val_accuracy: 0.7872
Epoch 6/50
17/17 [==============================] - 71s 4s/step - loss: 0.5541 - accuracy: 0.7564 - val_loss: 0.6178 - val_accuracy: 0.8298
Epoch 7/50
17/17 [==============================] - 70s 4s/step - loss: 0.5358 - accuracy: 0.7802 - val_loss: 0.6264 - val_accuracy: 0.7915
Epoch 8/50
17/17 [==

In [15]:
training_time = time.time() - start_time
print("\nTraining time: {:.2f} seconds".format(training_time))


Training time: 6914.98 seconds
